In [99]:
import torch
import torchvision
import numpy as np
torch.__version__

'1.4.0'

In [100]:
torchvision.__version__

'0.5.0'

### shape, size 一个是属性，一个是方法

In [101]:
a = torch.tensor([1,2,3])
a.shape, a.size(), type(a.shape), type(a.size())

(torch.Size([3]), torch.Size([3]), torch.Size, torch.Size)

In [102]:
a.shape[0], a.size()[0], a.numel()  #元素个数

(3, 3, 3)

In [103]:
na = np.array([[1,2,3], [4,5,6]])
nb = torch.tensor(na)
nc = torch.as_tensor(na)
nd = torch.from_numpy(na)

In [104]:
nb[:,1] = 55
na, nb

(array([[1, 2, 3],
        [4, 5, 6]]),
 tensor([[ 1, 55,  3],
         [ 4, 55,  6]], dtype=torch.int32))

In [105]:
nc[:,1] = 55
na, nc

(array([[ 1, 55,  3],
        [ 4, 55,  6]]),
 tensor([[ 1, 55,  3],
         [ 4, 55,  6]], dtype=torch.int32))

In [106]:
nd[:1] = 55
na, nd

(array([[55, 55, 55],
        [ 4, 55,  6]]),
 tensor([[55, 55, 55],
         [ 4, 55,  6]], dtype=torch.int32))

In [107]:
ne = [[1,2,3], [4,5,6]]
nf = torch.as_tensor(ne)
nf[:,1] = 55
ne, nf

([[1, 2, 3], [4, 5, 6]],
 tensor([[ 1, 55,  3],
         [ 4, 55,  6]]))

### 可见，as-tensor, from-numpy,都是改变numpy自身，直接tensor基本等于复制进去
### 但是对list进行as-tensor仍然是复制

In [108]:
a = torch.arange(1, 10)
a1 = a.view(3,3)
a.shape, a1.shape

(torch.Size([9]), torch.Size([3, 3]))

### 步长 stride

数据是在内存里单向连续分布，所以所谓的数组，矩阵，不过是告诉内存，跨多少个元素算一行，跨多少个元素算一列
对元素的view进行改变，比如转置，改变的貌似只是stride，内存本身没变

In [109]:
a.stride(), a1.stride()

((1,), (3, 1))

以上是正常数组，每三个元素一行，每个元素间隔一个，为一列

以下是转置后，每隔三个元素一列（1,4,7），但每隔一个元素就是1行了(1,2,3)

In [110]:
a2 = a1.permute(1, 0)
a2, a2.stride() 

(tensor([[1, 4, 7],
         [2, 5, 8],
         [3, 6, 9]]),
 (1, 3))

可以通过`contiguous`把它重新捋顺

In [111]:
if not a2.is_contiguous():
    a2 = a2.contiguous()
a2, a2.stride(), a2.is_contiguous()

(tensor([[1, 4, 7],
         [2, 5, 8],
         [3, 6, 9]]),
 (3, 1),
 True)

很显然，`contiguous`不改变数组表现形式，只改变内存方式

### 把数组传进去当形状 xxxx_like

In [112]:
value = torch.randn(3, 4)
zeros = torch.zeros_like(value)
ones  = torch.ones_like(value)
value, zeros, ones

(tensor([[ 0.5196, -0.1034,  1.1587, -0.8085],
         [ 0.9936,  0.1940, -1.6007,  0.2355],
         [-0.5803,  1.1558,  0.4824,  0.6533]]),
 tensor([[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]),
 tensor([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]))

In [113]:
s = torch.full((2,3), fill_value=3, dtype=torch.int32)  # FILL
s

tensor([[3, 3, 3],
        [3, 3, 3]], dtype=torch.int32)

In [114]:
a = torch.arange(24).reshape(4, 6)
b, c, d = a.chunk(3, dim=1)
e = torch.arange(25).reshape(5, 5)
f, g, h = e.chunk(3)
f, g, h

(tensor([[0, 1, 2, 3, 4],
         [5, 6, 7, 8, 9]]),
 tensor([[10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19]]),
 tensor([[20, 21, 22, 23, 24]]))

### 一些特殊索引（配合gather）

根据形状对应的列，取列里面的第几个元素
根据形状对应的行，取行里面的第几个元素

In [115]:
ids    = [
    [0,1,3,0,3,2],
    [0,1,2,0,3,2],
    [0,1,1,0,3,2]
]
indexs = torch.tensor(ids)
torch.gather(a, dim=0, index=indexs) # 不能传入ids，必须是tensor

# 按列取时列要写满，行无所谓

tensor([[ 0,  7, 20,  3, 22, 17],
        [ 0,  7, 14,  3, 22, 17],
        [ 0,  7,  8,  3, 22, 17]])

In [116]:
index2 = torch.tensor([
    [0,1,2],
    [5,5,5],
    [1,1,1],
    [2,3,4]
])
torch.gather(a, dim=1, index=index2)

# 按行取时行要写满，列无所谓

tensor([[ 0,  1,  2],
        [11, 11, 11],
        [13, 13, 13],
        [20, 21, 22]])

### 挤压 squeeze

把维度为1的通通压缩掉（即去掉只有一个子元素的的大括号
反向`unsqueeze`则是凭空加一个大括号, 加在哪一级由参数的`dim`决定

In [117]:
em = torch.tensor([[[[[1]]]]])
em.squeeze()

tensor(1)

In [118]:
torch.tensor([1,2,3]).unsqueeze(1) # 这样就堆叠起来了

tensor([[1],
        [2],
        [3]])

In [119]:
m = torch.arange(24).reshape(2,3,4)
m.unsqueeze(0).shape, m.unsqueeze(1).shape, m.unsqueeze(2).shape

(torch.Size([1, 2, 3, 4]), torch.Size([2, 1, 3, 4]), torch.Size([2, 3, 1, 4]))

In [120]:
m.permute(2,0,1)

tensor([[[ 0,  4,  8],
         [12, 16, 20]],

        [[ 1,  5,  9],
         [13, 17, 21]],

        [[ 2,  6, 10],
         [14, 18, 22]],

        [[ 3,  7, 11],
         [15, 19, 23]]])

### 特殊索引2 配合take

是把元素摊平按个数取的

In [121]:
index_v = torch.tensor([
    [5, 6, 7, 12],
    [0, 1, 2, 3]
])
(a*2).take(index_v)

tensor([[10, 12, 14, 24],
        [ 0,  2,  4,  6]])

### 条件取值 (类三元表达式）

In [122]:
torch_value1 = torch.arange(1, 10).view(3, 3)
torch_value2 = torch.full((3, 3), 88, dtype=torch.int64)

condition = torch.tensor([
    [True, False, True],
    [True, False, True],
    [True, False, True]
])

new_torch_value = torch.where(condition, torch_value1, torch_value2)  # if True torch_value1 else torch_value2
new_torch_value

tensor([[ 1, 88,  3],
        [ 4, 88,  6],
        [ 7, 88,  9]])

In [123]:
x = torch.randn(3, 3)
y = torch.zeros(3, 3)
condition = x > 0
z = torch.where(condition, x, y)
z

tensor([[0.0000, 0.0000, 0.1307],
        [0.0000, 1.1913, 0.0000],
        [0.0000, 0.0000, 0.5272]])

### Sigmoid

In [124]:
torch.tensor([0], dtype=torch.int64).sigmoid()

RuntimeError: "sigmoid_cpu" not implemented for 'Long'

In [125]:
torch.zeros(2).sigmoid()

tensor([0.5000, 0.5000])

In [126]:
torch_value = torch.arange(9).view(3, 3)
print(torch_value)
argmax_index = torch.argmax(torch_value, dim=1, keepdim=True)
print(argmax_index)

# 跟topk取top1差不多，但不能keepdim
print(torch_value.topk(1, dim=0)) 

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
tensor([[2],
        [2],
        [2]])
torch.return_types.topk(
values=tensor([[6, 7, 8]]),
indices=tensor([[2, 2, 2]]))


### topk

默认按行取每行最大值(dim=1), 按列取时返回每列里的索引
同时返出value和index, 比`argmax`多一项返回

In [127]:
print(a)
a.topk(3) # 如果有维度，则按维度取topk

tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]])


torch.return_types.topk(
values=tensor([[ 5,  4,  3],
        [11, 10,  9],
        [17, 16, 15],
        [23, 22, 21]]),
indices=tensor([[5, 4, 3],
        [5, 4, 3],
        [5, 4, 3],
        [5, 4, 3]]))

In [128]:
a.topk(2, dim=0)

torch.return_types.topk(
values=tensor([[18, 19, 20, 21, 22, 23],
        [12, 13, 14, 15, 16, 17]]),
indices=tensor([[3, 3, 3, 3, 3, 3],
        [2, 2, 2, 2, 2, 2]]))

### 特殊索引 boolean索引器

In [129]:
bi = torch.tensor([[True],[True],[False],[False]])
# b2 = torch.randn(4, 1)
# b2 = b2>0
a[bi]

# 索引器和数组形状不一样是不行的，但是numpy可以?

IndexError: The shape of the mask [4, 1] at index 1 does not match the shape of the indexed tensor [4, 6] at index 1

In [148]:
an = a.numpy()
bn = bi.numpy()
b2 = np.random.randn(4, 1)
b3 = np.array([[1],[2],[3],[4]])
an[b3>3,:]
# an[b2>0]
# c = np.arange(8).reshape(4,2)
# x = np.array([[0, 1], [1, 1], [2, 2], [0, 1], [1, 1], [2, 2]])
# x.sum(1)
# x[x.sum(1)<4]

# xy[label==i, 0]

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed